##Gaussian Mixture Model
モデルがわかっている場合には、その関数でフィットして、パラメータを決めていくだけです。ここでは、よくある例として、ピークが複数ある分布を、複数のガウシアンの足しあわせで近似する場合を考えます。
###1次元

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
#data7.txtは1カラムのデータ
file = open("data7.txt","r")
xL = []
for line in file:
    cols = line.split()
    x = float(cols[0])
    xL.append(x)

xs = np.array(xL)
X = xs[:, np.newaxis]            #列ベクトルに変換
plt.hist(X, 30, normed=True, histtype='stepfilled', alpha=0.4)   #ヒストグラムでプロット

3つのガウシアン分布でこの分布をフィットしてみる。(http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GMM.html)

In [ ]:
import numpy as np
from sklearn.mixture import GMM

#3つのガウス関数でフィットする。
model = GMM(3).fit(X)

x = np.linspace(-6, 6, 1000)
x = x[:, np.newaxis]            #列ベクトルに変換
y = np.exp(model.score(x))      #推定された分布関数(scoreは対数を返すので、指数関数をかけている)
plt.hist(X, 30, normed=True, histtype='stepfilled', alpha=0.4)
plt.plot(x,y)


3つの成分に分けてみます。

In [ ]:
import numpy as np
from sklearn.mixture import GMM

#3つのガウス関数でフィットする。
model = GMM(3).fit(X)

x = np.linspace(-6, 6, 1000)
x = x[:, np.newaxis]            #列ベクトルに変換
y = np.exp(model.score(x))      #推定された分布関数
w = model.score_samples(x)[1]   #各Gaussianの"重み"
y0 = w[:,0]*y
y1 = w[:,1]*y
y2 = w[:,2]*y

plt.hist(X, 30, normed=True, histtype='stepfilled', alpha=0.4)
plt.plot(x,y)
plt.plot(x,y0)
plt.plot(x,y1)
plt.plot(x,y2)

重みだけをプロットしてみます。

In [ ]:
import numpy as np
from sklearn.mixture import GMM

#3つのガウス関数でフィットする。
model = GMM(3).fit(X)

x = np.linspace(-6, 6, 1000)
x = x[:, np.newaxis]            #列ベクトルに変換
y = np.exp(model.score(x))      #推定された分布関数
w = model.score_samples(x)[1]   #各Gaussianの"重み"

plt.plot(x,w)


このグラフは何を意味しているのでしょうか。

実験で得られた分布が、仮に3つのガウシアンの足しあわせでできているとしましょう。

それはつまりこういうことです。測定している系は3つの独立な状態のいずれかに属しています。そして、状態1では、測定値は平均値-1のまわりで広い分布をもち、状態2では平均値0で分布し、状態3では平均値3のまわりで分布します。

測定する人間には、測定結果しかわからないので、系がどの状態にあるのかはわかりません。

もし、測定値が1だった時、系はどの状態にあると推定するのが一番合理的でしょうか。

もちろん、状態2にあると推定すべきでしょう。なぜなら、x=1では、3つの状態のなかで、状態2が一番確率が高いからです。これが賭け事だとすれば、あなたは状態2に賭けるべきなのは明らかです。

上のグラフの上側エンベロープは、あなたが賭けるべき状態を示しています。あるいは、一番もっともらしい、データの分類方法を示しているとも言えます。

ただし、一つ心配な点があります。この分布は、本当に3つのガウシアン関数の足しあわせで良いのでしょうか。

試しに、2つのガウシアンでフィットすると、あきらかに良くないことはわかります。

In [ ]:
import numpy as np
from sklearn.mixture import GMM

#3つのガウス関数でフィットする。
model = GMM(2).fit(X)

x = np.linspace(-6, 6, 1000)
x = x[:, np.newaxis]            #列ベクトルに変換
y = np.exp(model.score(x))      #推定された分布関数(scoreは対数を返すので、指数関数をかけている)
plt.hist(X, 30, normed=True, histtype='stepfilled', alpha=0.4)
plt.plot(x,y)

でも、4つのガウシアンでも問題ないようです。

In [ ]:
import numpy as np
from sklearn.mixture import GMM

#3つのガウス関数でフィットする。
model = GMM(4).fit(X)

x = np.linspace(-6, 6, 1000)
x = x[:, np.newaxis]            #列ベクトルに変換
y = np.exp(model.score(x))      #推定された分布関数(scoreは対数を返すので、指数関数をかけている)
plt.hist(X, 30, normed=True, histtype='stepfilled', alpha=0.4)
plt.plot(x,y)

では、最適な個数はいくつでしょうか。ここでも、あらかじめ3つの状態しかないことがわかっていれば良いのですが、それがわからない場合には困ってしまいます。ガウシアンの個数を増やせば、より忠実にヒストグラムのでこぼこに追従するようになりますが、それは本当に知りたい答とは限りません。

その場合も、確認用集合の考え方があてはめられるかもしれません。

###2次元以上
2次元以上の場合でも、ガウス分布の足しあわせに分解できます。

https://github.com/scikit-learn/scikit-learn/blob/master/examples/mixture/plot_gmm_pdf.py

In [ ]:
%matplotlib inline
#Modified from:
#https://github.com/scikit-learn/scikit-learn/blob/master/examples/mixture/plot_gmm_pdf.py

#サンプルデータの生成
import numpy as np
import matplotlib.pyplot as plt

n_samples = 300

# generate random sample, two components
np.random.seed(0)

# generate spherical data centered on (20, 20)
shifted_gaussian = np.random.randn(n_samples, 2) + np.array([20, 20])

# generate zero centered stretched Gaussian data
C = np.array([[0., -0.7], [3.5, .7]])
stretched_gaussian = np.dot(np.random.randn(n_samples, 2), C)

# concatenate the two datasets into the final training set
X_train = np.vstack([shifted_gaussian, stretched_gaussian])

plt.scatter(X_train[:, 0], X_train[:, 1], .8)

plt.axis('tight')
plt.show()

In [ ]:
#複数の二次元ガウス関数でフィットする。
from matplotlib.colors import LogNorm
from sklearn import mixture

# fit a Gaussian Mixture Model with two components
clf = mixture.GMM(n_components=2, covariance_type='full')
clf.fit(X_train)

# display predicted scores by the model as a contour plot
x = np.linspace(-20.0, 30.0)
y = np.linspace(-20.0, 40.0)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)[0]
Z = Z.reshape(X.shape)

CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 10))
CB = plt.colorbar(CS, shrink=0.8, extend='both')
plt.scatter(X_train[:, 0], X_train[:, 1], .8)

plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.show()